# Loading Data 

## Importing libraries and txt file

We can seperate the txt file into different features, since we don't know the physical qualities each index represent we will keep them as strings that correspond to each number.

In [1]:
import os
import tensorflow as tf
import datetime
import sys
from os import path
import numpy as np
import pandas as pd

